In [64]:
# imports
import pandas as pd
import numpy as np
import math
import Pyro4
import matplotlib.pyplot as plt
from nltk.stem.snowball import PorterStemmer
plt.switch_backend('agg')
% matplotlib inline
import seaborn as sns
# NLP tools
import nltk
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score
import gensim
from gensim.matutils import Sparse2Corpus, corpus2dense
from sklearn.decomposition import PCA
from gensim.corpora.dictionary import Dictionary
from gensim import corpora
from nltk.tokenize import sent_tokenize, word_tokenize
from __future__ import print_function, division
from gensim import corpora, models, similarities
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans, MiniBatchKMeans, AgglomerativeClustering
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics.pairwise as smp
from gensim.models import Word2Vec
from sklearn.decomposition import NMF
# Python 2 compatibility
from __future__ import print_function
# logging for gensim (set to INFO)
import logging
from nltk.corpus import stopwords
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [65]:
questions_train = pd.read_csv("/home/kalgi/train.csv")
question1 = questions_train.iloc[:,4]
question2 = questions_train['question2']
questions_all = pd.concat([question1, question2], axis=0).reset_index()
questions = (questions_all['question2'][:100000]).reset_index()
questions.dropna()
question_list = list(questions)

In [96]:
#tfidf & LSA
english_stemmer = PorterStemmer()
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([english_stemmer.stem(w) for w in analyzer(doc)])
tfidf = StemmedCountVectorizer(stop_words='english', token_pattern="\\b[a-z][a-z]+\\b", min_df=200, 
                      max_df=0.8, ngram_range=(1,3))
tfidf_vecs = tfidf.fit_transform(questions['question2'])
tfidf_corpus = Sparse2Corpus(tfidf_vecs.transpose())
id2word = dict((v, k) for k, v in tfidf.vocabulary_.items())
id2word = corpora.Dictionary.from_corpus(tfidf_corpus, 
                                         id2word=id2word)
lsi = models.LsiModel(tfidf_corpus, id2word=id2word, num_topics=200)
lsi_corpus = lsi[tfidf_corpus]
doc_vecs = [doc for doc in lsi_corpus]
questions['LSI_vecs'] = doc_vecs

In [85]:
# Let's take a look at how we did
for sim_doc_id, sim_score in sims[0:3]: 
    print("Score: " + str(sim_score))
    print("Document: " + questions['question2'][sim_doc_id])

TypeError: 'numpy.float32' object is not iterable

In [123]:
index = similarities.MatrixSimilarity(questions['LSI_vecs'],
                                      num_features=300)
sorted_sims = sorted(enumerate(index[questions['LSI_vecs'][95000]]), key=lambda item: -item[1])
print(sorted_sims[0:6])
print(questions['question2'][95000])
print(questions['question2'][5371])
print(questions['question2'][9690])
print(questions['question2'][10506])
print(questions['question2'][11116])
print(questions['question2'][13478])

[(5371, 1.0), (9690, 1.0), (10506, 1.0), (11116, 1.0), (13478, 1.0), (13597, 1.0)]
How do I avoid sleep so much?
How do I avoid sleep so much?
How should I avoid sleeping during lectures?
How would you overcome the fear of sleeping alone?
How do you treat sleep apnea without CPAP?
Do I have brain damage from sleep deprivation?


In [44]:
#similarities for tfidf-lsa
index = similarities.MatrixSimilarity(doc_vecs, 
                                      num_features=len(id2word))
with open('/dev/sims','w') as sims_output:
    for i, sims in enumerate(index):
        sims_with_labels = [(score, questions['question2'][j]) for j, score in enumerate(sims)]
        sorted_sims_with_labels = sorted(sims_with_labels, reverse=True)
        sims_output.write(str(sorted_sims_with_labels))
        sims_output.write('\n')

OSError: [Errno 28] No space left on device

In [ ]:
ng_lsi = corpus2dense(lsi_corpus, num_terms=100).transpose()
km = KMeans(n_clusters=10, random_state=1, n_jobs= 8)
km.fit_predict(ng_lsi)
questions['labels'] = km.labels_
Sil_coefs.append(metrics.silhouette_score(ng_lsi, labels, metric='euclidean'))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5), sharex=True)
k_clusters = range(3,10)
ax1.plot(k_clusters, Sil_coefs)
ax1.set_xlabel('number of clusters')
ax1.set_ylabel('silhouette coefficient')

In [ ]:
hierarchial = AgglomerativeClustering(n_clusters=10,
                                    linkage="average", affinity='cosine')
hierarchial.fit(ng_lsi)
questions['labels_hierarchial'] = hierarchial.labels_